# Total Value

In [15]:
from mantle import mantle
from env import (moneymarket_market_addr, pair_anc_ust_addr, moneymarket_overseer_addr, moneymarket_oracle_addr,
                 cw20_beth_addr, cw20_bluna_addr, lp_anc_ust_addr, staking_addr, gov_addr)
from ipynb.fs.defs.common_last_synced_height import last_synced_height
from ipynb.fs.defs.borrow_main_collateral_value import collaterals_total_value, strip_oracle_prices
from ipynb.fs.defs.common_token_balances import get_token_balances
from json import dumps
from numpy import array, sum

user_wallet_address = 'terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9'

block_height = last_synced_height()

token_balances = get_token_balances(user_wallet_address)

result = mantle(
    wasm_query={
        'anc_pool': {
            'contractAddress': pair_anc_ust_addr,
            'query': {
                'pool': {}
            }
        },
        'borrower_info': {
            'contractAddress': moneymarket_market_addr,
            'query': {
                'borrower_info': {
                    'borrower': user_wallet_address,
                    'block_height': block_height
                }
            }
        },
        'market_state': {
            'contractAddress': moneymarket_market_addr,
            'query': {
                'state': {}
            }
        },
        'overseer_collaterals': {
            'contractAddress': moneymarket_overseer_addr,
            'query': {
                'collaterals': {
                    'borrower': user_wallet_address
                }
            }
        },
        'oracle_prices': {
            'contractAddress': moneymarket_oracle_addr,
            'query': {
                'prices': {}
            }
        },
        'money_market_epoch_state': {
            'contractAddress': moneymarket_market_addr,
            'query': {
                'epoch_state': {
                    'block_height': block_height
                }
            }
        },
        'lp_balance': {
            'contractAddress': lp_anc_ust_addr,
            'query': {
                'balance': {
                    'address': user_wallet_address
                }
            }
        },
        'lp_staker_info': {
            'contractAddress': staking_addr,
            'query': {
                'staker_info': {
                    'staker': user_wallet_address,
                    'block_height': block_height
                }
            }
        },
        'gov_staker': {
            'contractAddress': gov_addr,
            'query': {
                'staker': {
                    'address': user_wallet_address
                }
            }
        }
    }
)

print(dumps(result, indent=2))
print(dumps(token_balances, indent=2))

{
  "anc_pool": {
    "assets": [
      {
        "amount": "9043337860977",
        "info": {
          "token": {
            "contract_addr": "terra1747mad58h0w4y589y3sk84r5efqdev9q4r02pc"
          }
        }
      },
      {
        "amount": "465335604633",
        "info": {
          "native_token": {
            "denom": "uusd"
          }
        }
      }
    ],
    "total_share": "1714145743838"
  },
  "borrower_info": {
    "borrower": "terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9",
    "interest_index": "1.066462506203885542",
    "loan_amount": "68838537",
    "pending_rewards": "208409.517505784802164225",
    "reward_index": "708.245274246842401894"
  },
  "gov_staker": {
    "balance": "5771554516",
    "locked_balance": [
      [
        37,
        {
          "balance": "100000000",
          "vote": "yes"
        }
      ],
      [
        53,
        {
          "balance": "10000000",
          "vote": "yes"
        }
      ],
      [
        54,
        {
      

In [16]:
anc_pool_size = float(result['anc_pool']['assets'][0]['amount'])
ust_pool_size = float(result['anc_pool']['assets'][1]['amount'])
anc_price = ust_pool_size / anc_pool_size

print('anc_price', anc_price)

anc_lp_share = float(result['anc_pool']['total_share'])

print('anc_lp_share (in LP)', anc_lp_share / 1000000)

ust_borrow_reward = float(result['borrower_info']['pending_rewards'])
ust_borrow_reward_value = ust_borrow_reward * anc_price

print('ust_borrow_reward (in ANC)', ust_borrow_reward / 1000000)
print('ust_borrow_reward_value (in UST)', ust_borrow_reward_value / 1000000)

# sum( [...collateral_locked_amount] * [...prices] )
total_collateral_value = collaterals_total_value(
    result['overseer_collaterals'],
    result['oracle_prices']
)

print('total_collateral_value (in UST)', total_collateral_value / 1000000)

loan_amount = float(result['borrower_info']['loan_amount'])

print('loan_amount', loan_amount)

exchange_rate = float(result['money_market_epoch_state']['exchange_rate'])
print('exchange_rate', exchange_rate)

oracle_prices = strip_oracle_prices(result['oracle_prices'])

holdings = array([
    token_balances['anc'],
    token_balances['beth'],
    token_balances['bluna']
])

holdings_prices = array([
    anc_price,
    float(
        next((item for item in oracle_prices if item['asset'] == cw20_beth_addr), None)['price']
    ),
    float(
        next((item for item in oracle_prices if item['asset'] == cw20_bluna_addr), None)['price']
    )
])

holdings_values = holdings * holdings_prices

print('holdings (in Token)', holdings / 1000000)
print('holdings_prices (in UST)', holdings_prices)
print('holdings_values (in UST)', holdings_values / 1000000)

user_lp_balance = float(result['lp_balance']['balance'])

anc_pool_asset = anc_pool_size * user_lp_balance / anc_lp_share
ust_pool_asset = ust_pool_size * user_lp_balance / anc_lp_share

print('anc_pool_asset (in ANC)', anc_pool_asset / 1000000)
print('ust_pool_asset (in UST)', ust_pool_asset / 1000000)

lp_value = ust_pool_size / anc_lp_share * 2
lp_staked = float(result['lp_staker_info']['bond_amount'])
lp_reward = float(result['lp_staker_info']['pending_reward'])
lp_staked_value = lp_staked * lp_value
lp_reward_value = lp_reward * anc_price

print('lp_value (in UST)', lp_value / 1000000)
print('lp_staked (in LP)', lp_staked / 1000000)
print('lp_staked_value (in UST)', lp_staked_value / 1000000)
print('lp_reward (in ANC)', lp_reward / 1000000)
print('lp_reward_value (in UST)', lp_reward_value / 1000000)

gov_staking = float(result['gov_staker']['balance'])
gov_staking_value = gov_staking * anc_price

print('gov_staking (in ANC)', gov_staking / 1000000)
print('gov_staking_value (in UST)', gov_staking_value / 1000000)

print('------------------------------------')

ust = token_balances['ust']
deposit = token_balances['aust'] * exchange_rate
borrowing = total_collateral_value - loan_amount + ust_borrow_reward_value
holdings = sum(holdings_values)
pool = (anc_pool_asset * anc_price) + ust_pool_asset
farming = lp_staked_value + lp_reward_value
govern = gov_staking_value

print('ust', ust / 1000000)
print('deposit', deposit / 1000000)
print('borrowing', borrowing / 1000000)
print('holdings', holdings / 1000000)
print('pool', pool / 1000000)
print('farming', farming / 1000000)
print('govern', govern / 1000000)

print('------------------------------------')

total_value = sum([ust, deposit, borrowing, holdings, pool, farming, govern])

print('total_value', total_value / 1000000)

anc_price 0.05145617821501223
anc_lp_share (in LP) 1714145.743838
ust_borrow_reward (in ANC) 0.20840951750578482
ust_borrow_reward_value (in UST) 0.010723957274482375
collateral_address ['terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l', 'terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x']
locked_amounts [  11350. 8600910.]
prices [3046.06378285   27.9222271 ]
locked_amounts (in UST) [ 34.57282394 240.15656233]
collateral_value (in UST) 274.72938626190114
total_collateral_value (in UST) 274.72938626190114
loan_amount 68838537.0
exchange_rate 1.0949956317264882
holdings (in Token) [9.04722839e+05 3.98865000e+00 8.71817950e+01]
holdings_prices (in UST) [5.14561782e-02 3.04606378e+03 2.79222271e+01]
holdings_values (in UST) [46553.57962457 12149.68230746  2434.30987938]
anc_pool_asset (in ANC) 26060.125067197536
ust_pool_asset (in UST) 1340.954439763224
lp_value (in UST) 5.429358691415656e-07
lp_staked (in LP) 910.0
lp_staked_value (in UST) 494.0716409188247
lp_reward (in ANC) 40.761487
lp_reward